In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np

In [2]:
df = pd.read_excel('../Data/Todo_esp/connectividad/Cobertura_banda_ancha_a_nivel_de_Entidad_Singular_de_Poblacion.xlsx', sheet_name='ESP')
df.drop(columns=['Habitantes'], inplace=True)

/home/pomilusky/anaconda3/envs/ironhack/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
# Vamos a tratar por provinicas, 
df = df[df['Comunidad Autónoma']== 'Castilla y León']

In [26]:
def cambio_enti(s):
    llista = s.lower().split(' ')
    n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
    s = ' '.join(n_llista)
    if '-' in s:
        llista = s.lower().split('-')
        n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
        s = ' '.join(n_llista)
    if '(la)' in s:
        s = 'La '+s.replace('(la)','').strip()
    if '(el)' in s:
        s = 'El '+s.replace('(el)','').strip()
    if '(los)' in s:
        s = 'Los '+s.replace('(los)','').strip()
    if '(las)' in s:
        s = 'Las '+s.replace('(las)','').strip()   

    if 'de La ' in s:
        s = s.replace('de La ', 'de la ')
    if 'de Las ' in s:
        s = s.replace('de Las ', 'de las ')
    if 'de Los ' in s:
        s = s.replace('de Los ', 'de los ')
    if ' Y ' in s:
        s = s.replace(' Y ', ' y ')
    if ' O ' in s:
        s = s.replace(' O ', ' o ')
    return s
    
def process_num(num):
    return float(re.sub(r"[^\w\s.]",'',num))
#cambio_enti('Quintana de Los Prados')

In [45]:
# Ahora que tenemos todas las Entidades singulares de poblacion vamos a buscar su ubicacion en wikipedia,
def scrap_wiki_ubi(x):
    provincia, municipio, entidad = x
    n_e = cambio_enti(entidad)
    try:
        URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}"
        #print(URL)
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'lxml') 
        bodies = soup.findAll('tbody')
        a = False
        for e in bodies:
            if 'Ubicación' in e.text:
                r_tabla = e.findAll('tr')
                a=True
                break

        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','-')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
        
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({provincia})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
    
        for i in r_tabla:
            if 'Ubicación' in i.text:
                here = i

        lat, lon = list(here)[1].text.split('/')[1].split(',')
        
        return process_num(lat), process_num(lon)
    except:
        # print(n_e)
        # i += 1
        # if i > 1000:
        #     i = 0
        #     print('ok')
        return np.nan

In [49]:
scrap_wiki_ubi(['Girona','Berga','Berga'])

https://es.wikipedia.org/wiki/Berga


(42.1, 1.8455555555556)

In [28]:
df['coord'] = df.apply(lambda x: scrap_wiki_ubi((x['Provincia'],x['Municipio'],x['Entidad Singular de Población'])), axis=1)

In [36]:
df.to_csv('../Data/castilla_leon/main.csv')